In [ ]:
%load_ext autoreload
%autoreload 2

from __future__ import division, print_function
%matplotlib inline

import os
import matplotlib.pyplot as plt

import numpy as np
np.set_printoptions(precision=4)
%cd -q ../test/


# SparseEdges : including prior information on edge co-occurrences

Natural images show a prototypical distribution of orientation. The goal of this notebook is to study how we could take advantage of this prior information to make the edge extraction even more efficient.

Indeed, oriented edges in images commonly occur in co-linear and co-circular arrangements, obeying the good continuation law of Gestalt psychology. The human visual system appears to exploit this property of images, with contour detection, line completion, and grouping performance well predicted by such an association field between edge elements (see Field, 93 and Geisler, 01). In this paper, we show that an association field of this type can be used to enhance the sparse representation of natural images. First, we define the [https://laurentperrinet.github.io/publication/perrinet-15-bicv](SparseLets) framework as an efficient representation of images based on a discrete wavelet transform. Second, we extract second-order information about edge co-occurrences from a set of images of natural scenes. Finally, we incorporate this prior information into our framework and show that it allows for the extraction of features relevant to natural scenes, like a round shape. This novel approach points the way to practical computer vision algorithms with human-like performance. 

Surprisingly, we will show that indeed we can make the extraction tuned to that statistics, but that the overall efficiency is not significantly improved.


This class exploits the [https://laurentperrinet.github.io/publication/perrinet-15-bicv](SparseEdges) package to provide with a sparse representation of edges in images.

This notebook reproduces the figure 14.4 of the following paper:

~~~~{.bibtex}
@inbook{Perrinet15bicv,
    author = {Perrinet, Laurent U.},
    booktitle = {Biologically-inspired Computer Vision},
    chapter = {13},
    citeulike-article-id = {13566753},
    editor = {Keil, Matthias and Crist\'{o}bal, Gabriel and Perrinet, Laurent U.},
    keywords = {anr-trax, bicv-sparse},
    posted-at = {2015-03-31 14:21:35},
    priority = {2},
    publisher = {Wiley, New-York},
    title = {Sparse models},
    year = {2015}
}
~~~~


More information is available @ http://nbviewer.ipython.org/github/bicv/SparseEdges/blob/master/SparseEdges.ipynb
Tests for the packages are available @ http://nbviewer.ipython.org/github/bicv/SparseEdges/blob/master/test-SparseEdges.ipynb.


In particular, this particular work was 

~~~~{.bibtex}

@inproceedings{Perrinet15eusipco,
    title = {Sparse Coding Of Natural Images Using A Prior On Edge {Co-Occurences}},
    author = {Perrinet, Laurent U. and Bednar, James A.},
    booktitle = {European Signal Processing Conference 2015 (EUSIPCO 2015)},
    abstract = {Oriented edges in images of natural scenes tend to be aligned in co-linear or co-circular arrangements, with lines and smooth curves more common than other possible arrangements of edges (the "good continuation law" of Gestalt psychology). The visual system appears to take advantage of this prior knowledge about natural images, with human contour detection and grouping performance well predicted by such an "association field" between edge elements. Geisler et al (2001) have estimated this prior information available to the visual system by extracting contours from a database of natural images, and showed that these statistics could predict behavioral data from humans in a line completion task. In this paper, we show that an association field of this type can be used for the sparse representation of natural images.},
    address = {Nice, France},
    citeulike-article-id = {13563378},
    keywords = {association, bicv-sparse, coding, connections, field, lateral, natural, scene, sparse, sparselets, statistics},
    month = aug,
    posted-at = {2015-03-27 10:56:57},
    priority = {2},
    year = {2015},
url = {http://ieeexplore.ieee.org/document/7362781/}
url = {http://dx.doi.org/10.1109/EUSIPCO.2015.7362781},
doi = {10.1109/EUSIPCO.2015.7362781},
}
~~~~


Table of content

* [A dipole as a second-order prior](#A-dipole-as-a-second-order-prior)
* [Prior on second-order distribution of edge co-occurences](#Prior-on-second-order-distribution-of-orientations)
* [Application to segmentation](#Application-to-segmentation)


## Initialization

In [ ]:
#! defining framework
#!-------------------
from SparseEdges import SparseEdgesWithDipole as SparseEdges
mp = SparseEdges('https://raw.githubusercontent.com/bicv/SparseEdges/master/default_param.py')
#mp = SparseEdgesWithDipole('../default_param.py')
mp.pe.datapath = 'database/'
#pe.N = 4
#pe.do_mask=False
#pe.MP_alpha=1.
#pe.do_whitening = False

print ('Range of spatial frequencies: ', mp.sf_0)

In [ ]:
print ('Range of angles: ', mp.theta*180./np.pi)

## A dipole as a second-order prior

### constructing the dipole step by step

Let's contruct a dipole step by step.

A dipole is defined by a reference edge, given by its position, orientationand scale, and by the probability associated to this reference.

We show it on a GoldenPyramid® :

In [ ]:
help(mp.golden_pyramid)

First, we define the relative difference of orientation $\psi$:

In [ ]:
def plot_dipole( w=.25, B_psi=.5, B_theta=.5, scale=1.5, epsilon=1.e-6):
    x, y, theta_edge, sf_0, C, phase = mp.pe.N_X/2, mp.pe.N_Y/2, mp.theta[-1], mp.sf_0[2], 1, 0.
    D = np.ones((mp.pe.N_X, mp.pe.N_Y, mp.pe.n_theta, mp.n_levels))
    for i_sf_0, sf_0_ in enumerate(mp.sf_0):
        for i_theta, theta_layer in enumerate(mp.theta):
            psi = np.arctan2(mp.Y-y, mp.X-x) - theta_layer #- theta_edge + theta/2 - np.pi/4
            psi = ((psi + np.pi/2  - np.pi/mp.pe.N_phi/2 ) % (np.pi)) - np.pi/2  + np.pi/mp.pe.N_phi/2
            D[:, :, i_theta, i_sf_0] = np.exp((np.cos(2*psi)-1.)/(B_psi**2))
    mp.golden_pyramid(D)
plot_dipole()

Now we add difference of orientation $\theta$

In [ ]:
def plot_dipole(w=mp.pe.dip_w, B_psi=mp.pe.dip_B_psi, B_theta=mp.pe.dip_B_psi, scale=mp.pe.dip_B_psi, epsilon=mp.pe.dip_epsilon):
    x, y, theta_edge, sf_0, C, phase = mp.pe.N_X/2, mp.pe.N_Y/2, mp.theta[-1], mp.sf_0[2], 1, 0.
    theta_edge = np.pi/2 - theta_edge
    #theta_edge = ((theta_edge + np.pi/2 - np.pi/mp.pe.n_theta/2)  % (np.pi) ) - np.pi/2  + np.pi/mp.pe.n_theta/2
    print (theta_edge*180/np.pi)
    D = np.ones((mp.pe.N_X, mp.pe.N_Y, mp.pe.n_theta, mp.n_levels))
    for i_sf_0, sf_0_ in enumerate(mp.sf_0):
        for i_theta, theta_layer in enumerate(mp.theta):
            theta_layer = np.pi/2 - theta_layer
            theta_layer = ((theta_layer + np.pi/2 - np.pi/mp.pe.n_theta/2)  % (np.pi) ) - np.pi/2  + np.pi/mp.pe.n_theta/2
            theta = theta_layer - theta_edge # angle between edge's orientation and the layer's one
            psi = np.arctan2(mp.Y-y, mp.X-x) - theta_edge -np.pi/2 - theta/2 #- np.pi/4
            #print i_theta, theta*180/np.pi
            #psi = ((psi + np.pi/2  - np.pi/mp.pe.N_phi/2 ) % (np.pi)) - np.pi/2  + np.pi/mp.pe.N_phi/2
            D[:, :, i_theta, i_sf_0] = np.exp((np.cos(2*(psi))-1.)/(B_psi**2))
            D[:, :, i_theta, i_sf_0] *= np.exp((np.cos(2*theta)-1.)/(B_theta**2))
    D -= D.mean()
    D /= np.abs(D).max()
    
    print (D.min(), D.mean(), D.max())
    mp.golden_pyramid(D)
plot_dipole()

Now we modulate it according to distance $d$ and scale to obtain the final dipole as implemented in ``SparseEdges``:

In [ ]:
def plot_dipole(w=mp.pe.dip_w, B_psi=mp.pe.dip_B_psi, B_theta=mp.pe.dip_B_psi, scale=mp.pe.dip_B_psi, epsilon=mp.pe.dip_epsilon):
    x, y, theta_edge, sf_0, C, phase = mp.pe.N_X/2., mp.pe.N_Y/2., mp.theta[-1], mp.sf_0[2], 1, 0.
    theta_edge = np.pi/2 - theta_edge
    print( theta_edge*180/np.pi)
    D = np.zeros((mp.pe.N_X, mp.pe.N_Y, mp.pe.n_theta, mp.n_levels))
    distance = np.sqrt(((mp.X-x)**2+(mp.Y-y)**2)/(mp.pe.N_X**2+mp.pe.N_Y**2))/w
    #print distance.max()
    neighborhood = np.exp(-distance**2)
    for i_sf_0, sf_0_ in enumerate(mp.sf_0):
        for i_theta, theta_layer in enumerate(mp.theta):
#                                if self.pe.scale_invariant: d *= np.sqrt(Sf_0[:, np.newaxis]*Sf_0[np.newaxis, :])#*np.sqrt(self.N_X)
            theta_layer = np.pi/2 - theta_layer
            theta_layer = ((theta_layer + np.pi/2 - np.pi/mp.pe.n_theta/2)  % (np.pi) ) - np.pi/2  + np.pi/mp.pe.n_theta/2
            theta = theta_layer - theta_edge # angle between edge's orientation and the layer's one
            psi = np.arctan2(mp.Y-y, mp.X-x) - theta_edge -np.pi/2 - theta/2 #- np.pi/4

            d = distance + epsilon
            D[:, :, i_theta, i_sf_0] = np.exp((np.cos(2*(psi))-1)/(B_psi**2 * d))#/(B_psi*np.sqrt(d))
            D[:, :, i_theta, i_sf_0] *= np.exp((np.cos(2*theta)-1)/(B_theta**2 * d))/(B_theta*np.sqrt(d))
#        D[:, :, :, i_sf_0] /= D[:, :, :, i_sf_0].mean()
#        D[:, :, :, i_sf_0] -= 1.
        D[:, :, :, i_sf_0] *= neighborhood[:, :, np.newaxis] * np.exp(-np.abs( np.log2(mp.sf_0[i_sf_0] / sf_0)) / scale)
#        D[:, :, :, i_sf_0] += 1.
    D -= D.mean()
    D /= np.abs(D).max()
    
    print (D.min(), D.mean(), D.max())

    mp.golden_pyramid(np.log2(1.+D))
plot_dipole()

In [ ]:

def plot_dipole(w=mp.pe.dip_w, B_psi=mp.pe.dip_B_psi, B_theta=mp.pe.dip_B_psi, scale=mp.pe.dip_B_psi, epsilon=mp.pe.dip_epsilon):
    mp.pe.dip_w = w
    mp.pe.dip_B_psi=B_psi
    mp.pe.dip_B_theta=B_theta
    mp.pe.dip_scale=scale
    mp.pe.dip_epsilon=epsilon
    logD = mp.dipole(np.array([mp.pe.N_X/2, mp.pe.N_Y/2, mp.theta[0], mp.sf_0[2], 1, 0.]))
    mp.golden_pyramid(logD)
#plot_dipole()
from IPython.html.widgets import interact
_ = interact(plot_dipole, w=(.0, .3, .0001), B_psi=(0.01, 2., 0.04), B_theta=(0.01, 2., 0.04), scale=(0.01, 20., .04), epsilon=(0, 1, 0.1))

co-circular axis of same angle are aligned.

### testing with a simple image of 2 edges

In [ ]:
import matplotlib
#matplotlib.rcParams.update({'font.size': 18, 'font.family': 'STIXGeneral', 'mathtext.fontset': 'stix'})
matplotlib.rcParams.update({'text.usetex': False})
%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib.pyplot as plt

import numpy as np
np.set_printoptions(precision=4)#, suppress=True)

In [ ]:
# mp = SparseEdgesWithDipole('https://raw.githubusercontent.com/bicv/SparseEdges/master/default_param.py')

from SparseEdges import SparseEdgesWithDipole
mp = SparseEdgesWithDipole('https://raw.githubusercontent.com/bicv/SparseEdges/master/default_param.py')

mp.pe.datapath = '../../SLIP/database/'

# filters in Fourier space
edge_in, edge_bis, edge_ter = [mp.pe.N_X*.5-10+42, mp.pe.N_Y*.5+10, 18, 2], [mp.pe.N_X*.5+42+10, mp.pe.N_Y*.5-10, 17, 2], [mp.pe.N_X*.25+42, mp.pe.N_Y*.25, 5, 2]
FT_lg_in = mp.loggabor(edge_in[0], edge_in[1], 
                       theta= mp.theta[edge_in[2]], B_theta=mp.pe.B_theta,
                       sf_0=mp.sf_0[edge_in[3]],  B_sf=mp.pe.B_sf)
FT_lg_bis = mp.loggabor(edge_bis[0], edge_bis[1], 
                        theta= mp.theta[edge_bis[2]], B_theta=mp.pe.B_theta,
                        sf_0=mp.sf_0[edge_bis[3]], B_sf=mp.pe.B_sf)
FT_lg_ter = mp.loggabor(edge_ter[0], edge_ter[1], 
                        theta= mp.theta[edge_ter[2]], B_theta=mp.pe.B_theta,
                        sf_0=mp.sf_0[edge_ter[3]], B_sf=mp.pe.B_sf)
# mixing both and shows one
FT_lg_ = 42. *  FT_lg_in + 35.*np.exp(1j*np.pi/4.) * FT_lg_bis + 40. *  FT_lg_ter
fig = mp.show_FT(FT_lg_)

In [ ]:
image = mp.invert(FT_lg_)
mp.pe.do_whitening = False
mp.pe.N = 5
mp.pe.MP_alpha = 1.

In [ ]:
z = np.ones((mp.pe.N_X, mp.pe.N_Y, mp.pe.n_theta, mp.n_levels))
for i_sf_0, sf_0_ in enumerate(mp.sf_0):
    for i_theta, theta in enumerate(mp.theta):
        params = {'sf_0':sf_0_, 'B_sf':mp.pe.B_sf, 'theta':theta, 'B_theta':mp.pe.B_theta}
        # loggabor takes as args: u, v, sf_0, B_sf, theta, B_theta)
        FT_lg = mp.loggabor(0, 0, **params)
        z[:, :, i_theta, i_sf_0] = np.absolute(mp.FTfilter(image, FT_lg, full=True))

fig = mp.golden_pyramid(z)

In [ ]:
print (' without second-order ')
mp.pe.eta_SO = 0.
edges, C_res = mp.run_mp(image, verbose=True)

In [ ]:
print (' with second-order ')
mp.pe.eta_SO = .3
edges, C_res = mp.run_mp(image, verbose=True)

In [ ]:
mp.pe.eta_SO = .01
self = mp
C = self.linear_pyramid(image)
logD = np.zeros((self.pe.N_X, self.pe.N_Y, self.pe.n_theta, self.n_levels))#, dtype=np.complex)
for i_edge in range(5):
    # MATCHING
    ind_edge_star = self.argmax(C * np.exp( self.pe.eta_SO * logD))

    coeff = self.pe.MP_alpha * np.absolute(C[ind_edge_star])
    # recording
    print ('Max activity  : ', np.absolute(C[ind_edge_star]), ' phase= ', np.angle(C[ind_edge_star], deg=True), ' deg,  @ ', ind_edge_star)
    edges[:, i_edge] = np.array([ind_edge_star[0]*1., ind_edge_star[1]*1.,
                             self.theta[ind_edge_star[2]],
                             self.sf_0[ind_edge_star[3]],
                             coeff, np.angle(C[ind_edge_star])])
    # PURSUIT
    if self.pe.eta_SO>0.: logD+= np.absolute(C[ind_edge_star]) * self.dipole(edges[:, i_edge])
    C = self.backprop(C, ind_edge_star)
    fig = self.golden_pyramid(C * np.exp( self.pe.eta_SO * logD))

## Dipole based on the measured statistics

In [ ]:
!ls cache_dir/edges/SparseLets*

In [ ]:
mp = SparseEdgesWithDipole('https://raw.githubusercontent.com/bicv/SparseEdges/master/default_param.py')
mp.pe.datapath = '../../SLIP/database/'


edgeslist = np.load('cache_dir/edges/SparseLets_serre07_distractors_edges.npy')
v_hist = mp.cohistedges(edgeslist, display=False)
v_hist.mean(), v_hist.shape
mp.pe.d_max = 50.
mp.cohistedges(edgeslist=None, v_hist=v_hist)
    

In [ ]:
def plot_dipole():
    D = np.ones((mp.pe.N_X, mp.pe.N_Y, mp.pe.n_theta, mp.n_levels))
    x, y, theta, sf_0, C, phase = mp.pe.N_X/2, mp.pe.N_Y/2, mp.theta[8], mp.sf_0[2], 1, 0.
    d = np.sqrt((mp.X-x)**2+(mp.Y-y)**2)/mp.pe.N_X
    #if mp.pe.scale_invariant: d *= np.sqrt(Sf_0[:, np.newaxis]*Sf_0[np.newaxis, :])#*np.sqrt(self.N_X)
    d *= mp.pe.d_width # distance in visual angle
    i_d = np.floor((d - mp.pe.d_min)/(mp.pe.d_max-mp.pe.d_min)*mp.pe.N_r)
    #print mp.pe.d_min, i_d.min(), i_d.mean(), i_d.max(), mp.pe.d_max
    i_d = i_d * (i_d>=0)
    #print mp.pe.d_min, i_d.min(), i_d.mean(), i_d.max(), mp.pe.d_max
    i_d = i_d * (i_d<mp.pe.N_r) + (mp.pe.N_r-1) * (i_d >= mp.pe.N_r)
    #print mp.pe.d_min, i_d.min(), i_d.mean(), i_d.max(), mp.pe.d_max

    for i_sf_0, sf_0_ in enumerate(mp.sf_0):
        loglevel = np.log2(sf_0_/sf_0)
        i_loglevel = np.floor((loglevel + mp.pe.loglevel_max)/(mp.pe.loglevel_max*2)*mp.pe.N_scale)* np.ones_like(i_d)
        print( i_loglevel.min(), i_loglevel.mean(), i_loglevel.max())

        for ii_theta, theta_ in enumerate(mp.theta):
            D_theta = theta - theta_ # angle between edge's orientation and the layer's one
            phi = np.arctan2(mp.Y-y, mp.X-x) - np.pi/2 - theta_ - D_theta/2
            # putting everything in the [-pi/2, pi/2[ quadrant
            theta = ((D_theta + np.pi/2 - np.pi/mp.pe.n_theta/2)  % (np.pi) ) - np.pi/2  + np.pi/mp.pe.n_theta/2
            phi = ((phi + np.pi/2  - np.pi/mp.pe.N_phi/2 ) % (np.pi)) - np.pi/2  + np.pi/mp.pe.N_phi/2
            # discretizing
            i_theta = np.floor((theta + np.pi/2)/np.pi*mp.pe.n_theta) * np.ones_like(i_d)
            i_theta = i_theta * (1-(i_theta == mp.pe.n_theta)) + 0. * (i_theta == mp.pe.n_theta)
            i_phi = np.floor((phi + np.pi/2)/np.pi*mp.pe.N_phi)
            #print 1-(i_phi == mp.pe.N_phi)
            i_phi = i_phi * (1-(i_phi == mp.pe.N_phi)) + 0. * (i_phi == mp.pe.N_phi)
            #print i_d.max(), i_phi.max(), i_theta.max(), i_loglevel.max()
            #print i_d.shape, i_phi.shape, i_theta.shape, i_loglevel.shape
            #print type(i_d), type(i_phi), type(i_theta), type(i_loglevel)
            #print ii_theta, i_sf_0, theta_
            D[:, :, ii_theta, i_sf_0] = v_hist[i_d.astype(int), i_phi.astype(int), i_theta.astype(int), i_loglevel.astype(int)]

    
    im_RGB, im_max = [], 0
    for i_sf_0, sf_0_ in enumerate(mp.sf_0):
        im_RGB_ = np.zeros((mp.pe.N_X, mp.pe.N_Y, 3))
        for i_theta, theta_ in enumerate(mp.theta):
            im_abs = np.absolute(D[:, :, i_theta, i_sf_0])
            RGB = np.array([.5*np.sin(2*theta_ + 2*i*np.pi/3)+.5 for i in range(3)])
            im_RGB_ += im_abs[:,:, np.newaxis] * RGB[np.newaxis, np.newaxis, :]

        im_max = np.max((im_max, im_RGB_.max()))
        im_RGB.append(im_RGB_)
       
    phi = (np.sqrt(5) +1.)/2. # golden number
    opts= {'vmin':0., 'vmax':1., 'interpolation':'nearest', 'origin':'upper'}
    fig_width = 16
    fig = plt.figure(figsize=(fig_width, fig_width/phi))
    xmin, ymin, size = 0, 0, 1.
    
    for i_sf_0, sf_0_ in enumerate(mp.sf_0):
        a = fig.add_axes((xmin/phi, ymin, size/phi, size), facecolor='w')
        a.axis(c='b', lw=0)
        plt.setp(a, xticks=[])
        plt.setp(a, yticks=[])
        a.imshow(im_RGB[i_sf_0]/im_max, **opts)
        a.grid(False)
        i_orientation = np.mod(i_sf_0, 4)
        if i_orientation==0:
            xmin += size
            ymin += size/phi**2
        elif i_orientation==1:
            xmin += size/phi**2
            ymin += -size/phi
        elif i_orientation==2:
            xmin += -size/phi
        elif i_orientation==3:
            ymin += size
        size /= phi 

plot_dipole()
#from IPython.html.widgets import interact
#_ = interact(plot_dipole, w=(pe.N_X*.01,pe.N_X,pe.N_X*.01), B_phi=(0.01, 2., 0.04), B_theta=(0.01, 2., 0.04), scale=(0.01, 2., 0.04))

In [ ]:
ind = np.array([[0, 0], [1, 2]])
v_hist[ind, ind, ind, ind]

## Application to segmentation

In [ ]:
mp.pe.N, N_circle, N_image = 1024, 36, 1
edgeslist = np.zeros((6, mp.pe.N+N_circle, N_image))
# random edges:
edgeslist[0, :mp.pe.N, :] = mp.pe.N_X * np.random.rand(mp.pe.N, N_image)
edgeslist[1, :mp.pe.N, :] = mp.pe.N_X * np.random.rand(mp.pe.N, N_image)
edgeslist[2, :mp.pe.N, :] = (np.pi* np.random.rand(mp.pe.N, N_image) ) % np.pi
edgeslist[3, :mp.pe.N, :] = 0.5 * (1- mp.pe.base_levels**(-mp.n_levels*(np.random.rand(mp.pe.N, N_image))))
edgeslist[4, :mp.pe.N, :] = 1.2*np.random.rand(mp.pe.N, N_image) * np.sign(np.random.randn(mp.pe.N, N_image))
edgeslist[5, :mp.pe.N, :] = 2*np.pi*np.random.rand(mp.pe.N, N_image)
# cocircular edges:
for i_N, angle in enumerate(np.linspace(0, 2*np.pi, N_circle)): #2*np.pi*np.random.rand(N_circle)):
    edgeslist[0, mp.pe.N + i_N, :] = mp.pe.N_X/2. - mp.pe.N_X/4.*np.sin(angle) + .0 * np.random.randn(N_image)
    edgeslist[1, mp.pe.N + i_N, :] = mp.pe.N_X/2. + mp.pe.N_X/4.*np.cos(angle) + .0 * np.random.randn(N_image)
    edgeslist[2, mp.pe.N + i_N, :] = (np.pi/2 + angle + .5*np.pi/180 * np.random.randn(N_image)) % np.pi
    edgeslist[3, mp.pe.N + i_N, :] = mp.sf_0[2] #0.03
    edgeslist[4, mp.pe.N + i_N, :] = .7 + .4*np.exp(np.cos(angle)/1.**2)

#print edgeslist.shape
#image_rec = mp.reconstruct(edgeslist[:,:,0])
#from pylab import imsave, gray
#imsave(fname='database/circle_in_noise.png', arr=image_rec, vmin=image_rec.min(), vmax=image_rec.max(), cmap=gray())

mp.pe.scale = 1.
mp.pe.line_width = 1.5
mp.pe.figsize_edges = 5

fig, a = mp.show_edges(edgeslist[:,:,0], image=image, color='toto', show_phase=False) #

Trying also with Geisler's figure:

In [ ]:
%%writefile experiment_secondorder.py
# -*- coding: utf8 -*-
from __future__ import division, print_function

# rm **/Geisler01Fig7A_secondorder*
# rm **/circle_in_noise_secondorder*

import os
import numpy as np
import matplotlib.pyplot as plt

from SparseEdges import SparseEdgesWithDipole as SparseEdges

#figpath = '../../CNRS/BICV-book/BICV_sparse/'
figpath = './'
fig_width_pt = 318.670  # Get this from LaTeX using \showthe\columnwidth
inches_per_pt = 1.0/72.27               # Convert pt to inches
fig_width = fig_width_pt*inches_per_pt  # width in inches


def init_mp():
    mp = SparseEdges('https://raw.githubusercontent.com/bicv/SparseEdges/master/default_param.py')
    mp.pe.datapath = 'database/'
    mp.pe.N = 60
    mp.pe.do_whitening = True
    mp.pe.MP_alpha = 1.
    mp.pe.figsize_edges = 12
    mp.pe.figsize_edges = .382 * fig_width
    mp.pe.scale = 1.3
    mp.pe.line_width = 1.5
    return mp

mp = init_mp()
eta_SO = 0.15

##############################################################################################################
figname = 'circle_in_noise'
if False: #os.path.isfile('database/' + figname + '.png'):
    # defining input image 
    from pylab import imread
    image = imread('database/' + figname + '.png').mean(axis=-1)
    print (image.mean(), image.std())
else:
    N, N_circle, N_image = 1024, 36, 1
    edgeslist = np.zeros((6, N+N_circle, N_image))
    np.random.seed(seed=42)
    # random edges:
    edgeslist[0, :N, :] = mp.pe.N_X * np.random.rand(N, N_image)
    edgeslist[1, :N, :] = mp.pe.N_X * np.random.rand(N, N_image)
    edgeslist[2, :N, :] = (np.pi* np.random.rand(N, N_image) ) % np.pi
    edgeslist[3, :N, :] = 0.5 * (1- mp.pe.base_levels**(-mp.n_levels*(np.random.rand(N, N_image))))
    edgeslist[4, :N, :] = 1.25*np.random.rand(N, N_image) * np.sign(np.random.randn(N, N_image))
    edgeslist[5, :N, :] = 2*np.pi*np.random.rand(N, N_image)
    # cocircular edges:
    for i_N, angle in enumerate(np.linspace(0, 2*np.pi, N_circle)): #2*np.pi*np.random.rand(N_circle)):
        edgeslist[0, N + i_N, :] = mp.pe.N_X/2. - mp.pe.N_X/4.*np.sin(angle) + .0 * np.random.randn(N_image)
        edgeslist[1, N + i_N, :] = mp.pe.N_X/2. + mp.pe.N_X/4.*np.cos(angle) + .0 * np.random.randn(N_image)
        edgeslist[2, N + i_N, :] = (np.pi/2 + angle + .5*np.pi/180 * np.random.randn(N_image)) % np.pi
        edgeslist[3, N + i_N, :] = mp.sf_0[2] #0.03
        edgeslist[4, N + i_N, :] = 1.1 + .15*np.exp(np.cos(angle)/1.**2)

    print (edgeslist.shape)
    image = mp.reconstruct(edgeslist[:,:,0])
    #from pylab import imsave, gray
    #imsave(fname='database/' + figname + '.png', arr=image, vmin=image.min(), vmax=image.max(), cmap=gray())

image = mp.normalize(image, center=True)
print (image.mean(), image.std())
v_max = 1.*image.max()
v_min = -v_max
##############################################################################################################
print( ' without edges ')
matname = os.path.join(mp.pe.matpath, figname + '_secondorder_A.npy')
try:
    fig, a = mp.show_edges(edges=np.zeros((6,0)), image=image, v_min=v_min, v_max=v_max, color='toto', show_phase=False) #
    if not(figpath==None): mp.savefig(fig, matname.replace('mat/', figpath).replace('.npy', ''))
    fig.show()
except:
    print ('File ', matname, ' is locked')
##############################################################################################################
print (' without second-order ')
matname = os.path.join(mp.pe.matpath, figname + '_secondorder_B.npy')
if not(os.path.isfile(matname)):
    if not(os.path.isfile(matname + '_lock')):
        file(matname + '_lock', 'w').close()
        mp.pe.eta_SO = 0.
        edges, C_res = mp.run_mp(image, verbose=True)
        np.save(matname, edges)
        os.remove(matname + '_lock')
try:
    edges = np.load(matname)
    fig, a = mp.show_edges(edges, image=image, v_min=v_min, v_max=v_max, color='toto', show_phase=False) #
    if not(figpath==None): mp.savefig(fig, matname.replace('mat/', figpath).replace('.npy', ''))
    fig.show()
except:
    print ('File ', matname, ' is locked')
##############################################################################################################
print (' with second-order ')
matname = os.path.join(mp.pe.matpath, figname + '_secondorder_C.npy')
if not(os.path.isfile(matname)):
    if not(os.path.isfile(matname + '_lock')):
        file(matname + '_lock', 'w').close()
        mp.pe.eta_SO = eta_SO
        edges, C_res = mp.run_mp(image, verbose=True)
        np.save(matname, edges)
        os.remove(matname + '_lock')
try:
    edges = np.load(matname)
    edges[4, :] *= -1 # turn red in blue...
    fig, a = mp.show_edges(edges, image=image, v_min=v_min, v_max=v_max, color='toto', show_phase=False) #
    fig.show()
    if not(figpath==None): mp.savefig(fig, matname.replace('mat/', figpath).replace('.npy', ''))

except:
    print ('File ', matname, ' is locked')
##############################################################################################################
if True:
    N_explore = 25
    base = 1.5
    ##############################################################################################################
    mp = init_mp()
    for mp.pe.eta_SO in np.logspace(-1., 1., N_explore, base=base)*eta_SO:
        matname = os.path.join(mp.pe.matpath, figname + '_secondorder_eta_SO_' + str(mp.pe.eta_SO).replace('.', '_') + '.npy')
        if not(os.path.isfile(matname)):
            if not(os.path.isfile(matname + '_lock')):
                file(matname + '_lock', 'w').close()
                edges, C_res = mp.run_mp(image, verbose=True)
                np.save(matname, edges)
                os.remove(matname + '_lock')
        try:
            edges = np.load(matname)
            edges[4, :] *= -1 # turn red in blue...
            fig, a = mp.show_edges(edges, image=image, v_min=v_min, v_max=v_max, color='toto', show_phase=False) #
            fig.savefig(matname.replace('mat/', mp.pe.figpath).replace('.npy', '.pdf'))
        except:
            print ('File ', matname, ' is locked')
        plt.close('all')
    ##############################################################################################################
    mp = init_mp()
    mp.pe.eta_SO = eta_SO
    for mp.pe.dip_epsilon in np.linspace(0, 1., N_explore):
        matname = os.path.join(mp.pe.matpath, figname + '_secondorder_dip_epsilon_' + str(mp.pe.dip_epsilon).replace('.', '_') + '.npy')
        if not(os.path.isfile(matname)):
            if not(os.path.isfile(matname + '_lock')):
                file(matname + '_lock', 'w').close()
                edges, C_res = mp.run_mp(image, verbose=True)
                np.save(matname, edges)
                os.remove(matname + '_lock')
        try:
            edges = np.load(matname)
            edges[4, :] *= -1 # turn red in blue...
            fig, a = mp.show_edges(edges, image=image, v_min=v_min, v_max=v_max, color='toto', show_phase=False) #
            fig.savefig(matname.replace('mat/', mp.pe.figpath).replace('.npy', '.pdf'))
        except:
            print ('File ', matname, ' is locked')
        plt.close('all')       
    ##############################################################################################################
    base = 2.
    ##############################################################################################################
    mp = init_mp()
    mp.pe.eta_SO = eta_SO
    for mp.pe.dip_w in np.logspace(-1., 1., N_explore, base=base)*mp.pe.dip_w:
        matname = os.path.join(mp.pe.matpath, figname + '_secondorder_dip_w_' + str(mp.pe.dip_w).replace('.', '_') + '.npy')
        if not(os.path.isfile(matname)):
            if not(os.path.isfile(matname + '_lock')):
                file(matname + '_lock', 'w').close()
                edges, C_res = mp.run_mp(image, verbose=True)
                np.save(matname, edges)
                os.remove(matname + '_lock')
        try:
            edges = np.load(matname)
            edges[4, :] *= -1 # turn red in blue...
            fig, a = mp.show_edges(edges, image=image, v_min=v_min, v_max=v_max, color='toto', show_phase=False) #
            fig.savefig(matname.replace('mat/', mp.pe.figpath).replace('.npy', '.pdf'))
        except:
            print ('File ', matname, ' is locked')
        plt.close('all')
    ##############################################################################################################
    mp = init_mp()
    mp.pe.eta_SO = eta_SO
    for mp.pe.dip_B_psi in np.logspace(-1., 1., N_explore, base=base)*mp.pe.dip_B_psi:
        matname = os.path.join(mp.pe.matpath, figname + '_secondorder_dip_B_psi_' + str(mp.pe.dip_B_psi).replace('.', '_') + '.npy')
        if not(os.path.isfile(matname)):
            if not(os.path.isfile(matname + '_lock')):
                file(matname + '_lock', 'w').close()
                edges, C_res = mp.run_mp(image, verbose=True)
                np.save(matname, edges)
                os.remove(matname + '_lock')
        try:
            edges = np.load(matname)
            edges[4, :] *= -1 # turn red in blue...
            fig, a = mp.show_edges(edges, image=image, v_min=v_min, v_max=v_max, color='toto', show_phase=False) #
            fig.savefig(matname.replace('mat/', mp.pe.figpath).replace('.npy', '.pdf'))
        except:
            print ('File ', matname, ' is locked')
        plt.close('all')
    ##############################################################################################################
    mp = init_mp()
    mp.pe.eta_SO = eta_SO
    for mp.pe.dip_B_theta in np.logspace(-1., 1., N_explore, base=base)*mp.pe.dip_B_theta:
        matname = os.path.join(mp.pe.matpath, figname + '_secondorder_dip_B_theta_' + str(mp.pe.dip_B_theta).replace('.', '_') + '.npy')
        if not(os.path.isfile(matname)):
            if not(os.path.isfile(matname + '_lock')):
                file(matname + '_lock', 'w').close()
                edges, C_res = mp.run_mp(image, verbose=True)
                np.save(matname, edges)
                os.remove(matname + '_lock')
        try:
            edges = np.load(matname)
            edges[4, :] *= -1 # turn red in blue...
            fig, a = mp.show_edges(edges, image=image, v_min=v_min, v_max=v_max, color='toto', show_phase=False) #
            fig.savefig(matname.replace('mat/', mp.pe.figpath).replace('.npy', '.pdf'))
        except:
            print( 'File ', matname, ' is locked')
        plt.close('all')
    ##############################################################################################################
    mp = init_mp()
    mp.pe.eta_SO = eta_SO
    for mp.pe.dip_scale in np.logspace(-1., 1., N_explore, base=base)*mp.pe.dip_scale:
        matname = os.path.join(mp.pe.matpath, figname + '_secondorder_dip_scale_' + str(mp.pe.dip_scale).replace('.', '_') + '.npy')
        if not(os.path.isfile(matname)):
            if not(os.path.isfile(matname + '_lock')):
                file(matname + '_lock', 'w').close()
                edges, C_res = mp.run_mp(image, verbose=True)
                np.save(matname, edges)
                os.remove(matname + '_lock')
        try:
            edges = np.load(matname)
            edges[4, :] *= -1 # turn red in blue...
            fig, a = mp.show_edges(edges, image=image, v_min=v_min, v_max=v_max, color='toto', show_phase=False) #
            fig.savefig(matname.replace('mat/', mp.pe.figpath).replace('.npy', '.pdf'))
            fig
        except:
            print ('File ', matname, ' is locked')
        plt.close('all')
    ##############################################################################################################


In [ ]:
%run experiment_secondorder.py

## showing what happens

In [ ]:
import matplotlib
#matplotlib.rcParams.update({'font.size': 18, 'font.family': 'STIXGeneral', 'mathtext.fontset': 'stix'})
matplotlib.rcParams.update({'text.usetex': False})
%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib.pyplot as plt

import numpy as np
np.set_printoptions(precision=4)#, suppress=True)


In [ ]:
mp = SparseEdgesWithDipole('https://raw.githubusercontent.com/bicv/SparseEdges/master/default_param.py')
mp.pe.datapath = 'database/'
mp.pe.MP_alpha = 1.
mp.pe.N = 210
mp.pe.N = 105
mp.pe.do_whitening = False
#pe.do_whitening = True


#figpath = '../../CNRS/BICV-book/BICV_INT/BICV-sparse/'
figpath = './'
FORMATS = ['pdf', 'eps']
fig_width_pt = 318.670  # Get this from LaTeX using \showthe\columnwidth
inches_per_pt = 1.0/72.27               # Convert pt to inches
fig_width = fig_width_pt*inches_per_pt  # width in inches
mp.pe.figsize_edges = 12
mp.pe.figsize_edges = .382 * fig_width
mp.pe.scale = 1.3
mp.pe.line_width = 1.5


# defining input image 
from pylab import imread
image = imread('https://raw.githubusercontent.com/bicv/SparseEdges/master/database/circle_in_noise.png').mean(axis=-1)
print( image.mean(), image.std())

image = mp.normalize(image, center=True)
print (image.mean(), image.std())
v_max = 1.*image.max()
v_min = -v_max

print( mp.n_levels, mp.sf_0)
mp.pe.eta_SO = 0.15

In [ ]:
C = mp.linear_pyramid(image)

In [ ]:
fig = mp.golden_pyramid(C)

In [ ]:
self = mp
ind_edge_star = self.argmax(C)
coeff = self.pe.MP_alpha * np.absolute(C[ind_edge_star])
print ('Max activity  : ', np.absolute(C[ind_edge_star]), ' phase= ', np.angle(C[ind_edge_star], deg=True), ' deg,  @ ', ind_edge_star)

In [ ]:
edges = np.zeros((6, self.pe.N))
i_edge = 0
logD = np.zeros((self.pe.N_X, self.pe.N_Y, self.pe.n_theta, self.n_levels))#, dtype=np.complex)
edges[:, i_edge] = np.array([ind_edge_star[0]*1., ind_edge_star[1]*1.,
                                         self.theta[ind_edge_star[2]],
                                         self.sf_0[ind_edge_star[3]],
                                         coeff, np.angle(C[ind_edge_star])])
logD+= np.absolute(C[ind_edge_star]) * self.dipole(edges[:, i_edge])
fig = self.golden_pyramid(logD)

In [ ]:
C = self.backprop(C, ind_edge_star)
fig = self.golden_pyramid(C + self.pe.eta_SO * logD)

In [ ]:
C = mp.linear_pyramid(image)
logD = np.zeros((self.pe.N_X, self.pe.N_Y, self.pe.n_theta, self.n_levels))#, dtype=np.complex)
for i_edge in range(60):
    # MATCHING
    ind_edge_star = self.argmax(C * np.exp( self.pe.eta_SO * logD))

    coeff = self.pe.MP_alpha * np.absolute(C[ind_edge_star])
    # recording
    print ('Max activity  : ', np.absolute(C[ind_edge_star]), ' phase= ', np.angle(C[ind_edge_star], deg=True), ' deg,  @ ', ind_edge_star)
    edges[:, i_edge] = np.array([ind_edge_star[0]*1., ind_edge_star[1]*1.,
                             self.theta[ind_edge_star[2]],
                             self.sf_0[ind_edge_star[3]],
                             coeff, np.angle(C[ind_edge_star])])
    # PURSUIT
    if self.pe.eta_SO>0.: logD+= np.absolute(C[ind_edge_star]) * self.dipole(edges[:, i_edge])
    C = self.backprop(C, ind_edge_star)
    if i_edge%10==0 : fig = self.golden_pyramid(C * np.exp( self.pe.eta_SO * logD))

In [ ]:
image_rec = mp.reconstruct(edges)
print ('RMSE-W = ', ((mp.whitening(image)-image_rec)**2).sum()/((mp.whitening(image))**2).sum())

mp.pe.scale = 1.
mp.pe.line_width = 1.5
mp.pe.figsize_edges = 5


fig, a = mp.show_edges(edges, image=image_rec*1.)


In [ ]:
list_of_number_of_edge = range(0, 70, 8)

figname = 'circle_in_noise'
from pylab import imread
image = imread('https://raw.githubusercontent.com/bicv/SparseEdges/master/database/' + figname + '.png').mean(axis=-1)

edges_without = np.load('cache_dir/' + figname + '_secondorder_B.npy')
edges_without[4, :] *= -1
edges_with= np.load('cache_dir/' + figname + '_secondorder_C.npy')


mp.pe.scale = 1.
mp.pe.line_width = 5.5
mp.pe.figsize_edges = 5


fig_width = 14

for number_of_edge in list_of_number_of_edge:
    fig, axs = plt.subplots(1, 2, figsize=(fig_width, fig_width/2))
    fig, axs[0] = mp.show_edges(edges_without[:, :number_of_edge], image=image, fig=fig, a=axs[0], color='red', show_phase=False) #
    axs[0].text(14, 22, 'N=%d' % number_of_edge, color='red', fontsize=32)
    fig, axs[1] = mp.show_edges(edges_with[:, :number_of_edge], image=image, fig=fig, a=axs[1], color='red', show_phase=False) #
    plt.tight_layout()
    mp.savefig(fig, 'MP_prior_' + str(number_of_edge), figpath='../figures', formats=['png'])


## some book keeping for the notebook

In [ ]:
import watermark

In [ ]:
%watermark

In [ ]:
import version_information
%version_information numpy, scipy, matplotlib, sympy

In [ ]:
%cd -q ../notebooks